In [ ]:
#Use Random Forest to prepare a model on fraud data 
#treating those who have taxable_income <= 30000 as "Risky" and others are "Good"

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

#Fraud Check

In [ ]:
Fc=pd.read_csv('/content/sample_data/Fraud_check (1).csv')
Fc

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


###EDA

In [ ]:
Fc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    object
 1   Marital.Status   600 non-null    object
 2   Taxable.Income   600 non-null    int64 
 3   City.Population  600 non-null    int64 
 4   Work.Experience  600 non-null    int64 
 5   Urban            600 non-null    object
dtypes: int64(3), object(3)
memory usage: 28.2+ KB


In [ ]:
Fc.isnull().sum()

Undergrad          0
Marital.Status     0
Taxable.Income     0
City.Population    0
Work.Experience    0
Urban              0
dtype: int64

In [ ]:
Fc.describe()

,Taxable.Income,City.Population,Work.Experience
count,600.000000,600.000000,600.000000
mean,55208.375000,108747.368333,15.558333
std,26204.827597,49850.075134,8.842147
min,10003.000000,25779.000000,0.000000
25%,32871.500000,66966.750000,8.000000
50%,55074.500000,106493.500000,15.000000
75%,78611.750000,150114.250000,24.000000
max,99619.000000,199778.000000,30.000000


In [ ]:
Fc=pd.get_dummies(Fc)
Fc

,Taxable.Income,City.Population,Work.Experience,Undergrad_NO,Undergrad_YES,Marital.Status_Divorced,Marital.Status_Married,Marital.Status_Single,Urban_NO,Urban_YES
0,68833,50047,10,1,0,0,0,1,0,1
1,33700,134075,18,0,1,1,0,0,0,1
2,36925,160205,30,1,0,0,1,0,0,1
3,50190,193264,15,0,1,0,0,1,0,1
4,81002,27533,28,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...
595,76340,39492,7,0,1,1,0,0,0,1
596,69967,55369,2,0,1,1,0,0,0,1
597,47334,154058,0,1,0,1,0,0,0,1
598,98592,180083,17,0,1,0,1,0,1,0


In [ ]:
Fc['Taxable.Income'].max()

99619

In [ ]:
Fc['Taxable.Income'].min()

10003

In [ ]:
Fc['Tax']=pd.cut(Fc['Taxable.Income'], bins = [10002,30000,99620], labels = ["Risky", "Good"])
Fc

,Taxable.Income,City.Population,Work.Experience,Undergrad_NO,Undergrad_YES,Marital.Status_Divorced,Marital.Status_Married,Marital.Status_Single,Urban_NO,Urban_YES,Tax
0,68833,50047,10,1,0,0,0,1,0,1,Good
1,33700,134075,18,0,1,1,0,0,0,1,Good
2,36925,160205,30,1,0,0,1,0,0,1,Good
3,50190,193264,15,0,1,0,0,1,0,1,Good
4,81002,27533,28,1,0,0,1,0,1,0,Good
...,...,...,...,...,...,...,...,...,...,...,...
595,76340,39492,7,0,1,1,0,0,0,1,Good
596,69967,55369,2,0,1,1,0,0,0,1,Good
597,47334,154058,0,1,0,1,0,0,0,1,Good
598,98592,180083,17,0,1,0,1,0,1,0,Good


In [ ]:
array=Fc.values
array

array([[68833, 50047, 10, ..., 0, 1, 'Good'],
       [33700, 134075, 18, ..., 0, 1, 'Good'],
       [36925, 160205, 30, ..., 0, 1, 'Good'],
       ...,
       [47334, 154058, 0, ..., 0, 1, 'Good'],
       [98592, 180083, 17, ..., 1, 0, 'Good'],
       [96519, 158137, 16, ..., 1, 0, 'Good']], dtype=object)

In [ ]:
S=array[:,1:10]
S

array([[50047, 10, 1, ..., 1, 0, 1],
       [134075, 18, 0, ..., 0, 0, 1],
       [160205, 30, 1, ..., 0, 0, 1],
       ...,
       [154058, 0, 1, ..., 0, 0, 1],
       [180083, 17, 0, ..., 0, 1, 0],
       [158137, 16, 1, ..., 0, 1, 0]], dtype=object)

In [ ]:
T=array[:,10]
T

array(['Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good',
       'Good', 'Good', 'Risky', 'Good', 'Risky', 'Good', 'Good', 'Good',
       'Risky', 'Good', 'Good', 'Risky', 'Good', 'Risky', 'Good', 'Good',
       'Good', 'Good', 'Good', 'Good', 'Risky', 'Good', 'Good', 'Good',
       'Good', 'Good', 'Risky', 'Good', 'Risky', 'Good', 'Risky', 'Good',
       'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good',
       'Risky', 'Good', 'Good', 'Good', 'Good', 'Risky', 'Good', 'Risky',
       'Good', 'Good', 'Risky', 'Good', 'Good', 'Risky', 'Good', 'Good',
       'Good', 'Good', 'Good', 'Good', 'Risky', 'Good', 'Good', 'Good',
       'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Risky',
       'Good', 'Good', 'Good', 'Good', 'Good', 'Risky', 'Good', 'Risky',
       'Good', 'Risky', 'Good', 'Good', 'Good', 'Risky', 'Risky', 'Good',
       'Risky', 'Good', 'Risky', 'Good', 'Good', 'Good', 'Good', 'Good',
       'Good', 'Good', 'Risky', 'Good', 'Good', 'Good'

###K Fold Cross Validation

In [ ]:
kfold=KFold(n_splits=10,shuffle=True)

In [ ]:
model1=RandomForestClassifier(n_estimators=100,max_features=5)
model1

RandomForestClassifier(max_features=5)

In [ ]:
fit1=model1.fit(S,T)
pred=fit1.predict(S)
pred

array(['Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good',
       'Good', 'Good', 'Risky', 'Good', 'Risky', 'Good', 'Good', 'Good',
       'Risky', 'Good', 'Good', 'Risky', 'Good', 'Risky', 'Good', 'Good',
       'Good', 'Good', 'Good', 'Good', 'Risky', 'Good', 'Good', 'Good',
       'Good', 'Good', 'Risky', 'Good', 'Risky', 'Good', 'Risky', 'Good',
       'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good',
       'Risky', 'Good', 'Good', 'Good', 'Good', 'Risky', 'Good', 'Risky',
       'Good', 'Good', 'Risky', 'Good', 'Good', 'Risky', 'Good', 'Good',
       'Good', 'Good', 'Good', 'Good', 'Risky', 'Good', 'Good', 'Good',
       'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Good', 'Risky',
       'Good', 'Good', 'Good', 'Good', 'Good', 'Risky', 'Good', 'Risky',
       'Good', 'Risky', 'Good', 'Good', 'Good', 'Risky', 'Risky', 'Good',
       'Risky', 'Good', 'Risky', 'Good', 'Good', 'Good', 'Good', 'Good',
       'Good', 'Good', 'Risky', 'Good', 'Good', 'Good'

In [ ]:
result=cross_val_score(model1,S,T,cv=kfold)
result

array([0.78333333, 0.76666667, 0.73333333, 0.7       , 0.8       ,
       0.71666667, 0.75      , 0.71666667, 0.71666667, 0.71666667])

In [ ]:
result.mean()*100

74.0

#Company Data

In [ ]:
#A cloth manufacturing company is interested to know about the segment or attributes causes high sale. 
#Approach - A Random Forest can be built with target variable Sales (we will first convert it in categorical variable) 
#& all other variable will be independent in the analysis.  

In [ ]:
company=pd.read_csv('/content/sample_data/Company_Data (1).csv')
company

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,Good,33,14,Yes,Yes
396,6.14,139,23,3,37,120,Medium,55,11,No,Yes
397,7.41,162,26,12,368,159,Medium,40,18,Yes,Yes
398,5.94,100,79,7,284,95,Bad,50,12,Yes,Yes


###EDA

In [ ]:
company.info

<bound method DataFrame.info of      Sales  CompPrice  Income  Advertising  Population  Price ShelveLoc  Age  \
0     9.50        138      73           11         276    120       Bad   42   
1    11.22        111      48           16         260     83      Good   65   
2    10.06        113      35           10         269     80    Medium   59   
3     7.40        117     100            4         466     97    Medium   55   
4     4.15        141      64            3         340    128       Bad   38   
..     ...        ...     ...          ...         ...    ...       ...  ...   
395  12.57        138     108           17         203    128      Good   33   
396   6.14        139      23            3          37    120    Medium   55   
397   7.41        162      26           12         368    159    Medium   40   
398   5.94        100      79            7         284     95       Bad   50   
399   9.71        134      37            0          27    120      Good   49   

     Ed

In [ ]:
company.isnull().sum()

Sales          0
CompPrice      0
Income         0
Advertising    0
Population     0
Price          0
ShelveLoc      0
Age            0
Education      0
Urban          0
US             0
dtype: int64

In [ ]:
company.describe()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,7.496325,124.975000,68.657500,6.635000,264.840000,115.795000,53.322500,13.900000
std,2.824115,15.334512,27.986037,6.650364,147.376436,23.676664,16.200297,2.620528
min,0.000000,77.000000,21.000000,0.000000,10.000000,24.000000,25.000000,10.000000
25%,5.390000,115.000000,42.750000,0.000000,139.000000,100.000000,39.750000,12.000000
50%,7.490000,125.000000,69.000000,5.000000,272.000000,117.000000,54.500000,14.000000
75%,9.320000,135.000000,91.000000,12.000000,398.500000,131.000000,66.000000,16.000000
max,16.270000,175.000000,120.000000,29.000000,509.000000,191.000000,80.000000,18.000000


In [ ]:
array=company.values
array

array([[9.5, 138, 73, ..., 17, 'Yes', 'Yes'],
       [11.22, 111, 48, ..., 10, 'Yes', 'Yes'],
       [10.06, 113, 35, ..., 12, 'Yes', 'Yes'],
       ...,
       [7.41, 162, 26, ..., 18, 'Yes', 'Yes'],
       [5.94, 100, 79, ..., 12, 'Yes', 'Yes'],
       [9.71, 134, 37, ..., 16, 'Yes', 'Yes']], dtype=object)

In [ ]:
X=array[:,0:6]
X

array([[9.5, 138, 73, 11, 276, 120],
       [11.22, 111, 48, 16, 260, 83],
       [10.06, 113, 35, 10, 269, 80],
       ...,
       [7.41, 162, 26, 12, 368, 159],
       [5.94, 100, 79, 7, 284, 95],
       [9.71, 134, 37, 0, 27, 120]], dtype=object)

In [ ]:
Y=array[:,6]
Y

array(['Bad', 'Good', 'Medium', 'Medium', 'Bad', 'Bad', 'Medium', 'Good',
       'Medium', 'Medium', 'Bad', 'Good', 'Medium', 'Good', 'Good',
       'Medium', 'Good', 'Good', 'Good', 'Medium', 'Medium', 'Good',
       'Medium', 'Medium', 'Bad', 'Good', 'Good', 'Medium', 'Bad', 'Bad',
       'Good', 'Medium', 'Good', 'Good', 'Medium', 'Medium', 'Good',
       'Medium', 'Medium', 'Bad', 'Bad', 'Bad', 'Medium', 'Medium',
       'Medium', 'Bad', 'Medium', 'Bad', 'Bad', 'Good', 'Bad', 'Bad',
       'Bad', 'Medium', 'Medium', 'Medium', 'Medium', 'Bad', 'Bad',
       'Medium', 'Bad', 'Medium', 'Bad', 'Medium', 'Medium', 'Medium',
       'Medium', 'Medium', 'Good', 'Medium', 'Good', 'Medium', 'Medium',
       'Good', 'Medium', 'Bad', 'Medium', 'Medium', 'Medium', 'Bad',
       'Bad', 'Good', 'Good', 'Bad', 'Bad', 'Medium', 'Medium', 'Good',
       'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Bad',
       'Medium', 'Good', 'Bad', 'Good', 'Bad', 'Medium', 'Medium',
       'Medium

###K Fold Cross Validation

In [ ]:
kfold=KFold(n_splits=8,shuffle=True)

In [ ]:
model=RandomForestClassifier(n_estimators=100,max_features=4)
model

RandomForestClassifier(max_features=4)

In [ ]:
fit=model.fit(X,Y)

In [ ]:
pred=fit.predict(X)
pred

array(['Bad', 'Good', 'Medium', 'Medium', 'Bad', 'Bad', 'Medium', 'Good',
       'Medium', 'Medium', 'Bad', 'Good', 'Medium', 'Good', 'Good',
       'Medium', 'Good', 'Good', 'Good', 'Medium', 'Medium', 'Good',
       'Medium', 'Medium', 'Bad', 'Good', 'Good', 'Medium', 'Bad', 'Bad',
       'Good', 'Medium', 'Good', 'Good', 'Medium', 'Medium', 'Good',
       'Medium', 'Medium', 'Bad', 'Bad', 'Bad', 'Medium', 'Medium',
       'Medium', 'Bad', 'Medium', 'Bad', 'Bad', 'Good', 'Bad', 'Bad',
       'Bad', 'Medium', 'Medium', 'Medium', 'Medium', 'Bad', 'Bad',
       'Medium', 'Bad', 'Medium', 'Bad', 'Medium', 'Medium', 'Medium',
       'Medium', 'Medium', 'Good', 'Medium', 'Good', 'Medium', 'Medium',
       'Good', 'Medium', 'Bad', 'Medium', 'Medium', 'Medium', 'Bad',
       'Bad', 'Good', 'Good', 'Bad', 'Bad', 'Medium', 'Medium', 'Good',
       'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Bad',
       'Medium', 'Good', 'Bad', 'Good', 'Bad', 'Medium', 'Medium',
       'Medium

In [ ]:
results=cross_val_score(model,X,Y,cv=kfold)
results

array([0.56, 0.72, 0.64, 0.56, 0.68, 0.6 , 0.72, 0.7 ])

In [ ]:
results.mean()*100

64.75000000000001